In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Common Engine Interface

Throughout the different chapters, multiple chess engines using different algorithms will be developed. In this chapter some common code needed by all engines will be developed.

In general, all engines will need to represent a move along with a score. Therefore a simple data class `ScoredMove` is introduced. By using the `dataclass` decorater with `order=True` python comparison methods, 
such as `__lt__()`, will be auto-generated.
This allows to compare two scored moves or sort a list of scored moves.
The comparison is only based on the score as the move field has `compare=False`. 

In [ ]:
from dataclasses import dataclass, field
import chess
from chess.engine import Score


@dataclass(order=True)
class ScoredMove:
    """Class for representing a move along with a score."""
    score: Score
    move: chess.Move = field(compare=False)

To simplify writing generic code for playing a chess game with different engines,
all engines will implement a common interface. 
The interface `Engine`,
which is highly inspired by `chess.engine`,
has two functions: `play` and `analyse`.

The `play` function takes the current board as a parameter 
and returns a `chess.engine.PlayResult` object, 
which contains the next move 
and information if the engine offered a draw or resigned. 

The `analyse` function takes the board as a parameter as well 
and returns a list of `ScoredMove` objects.


In [ ]:
import chess.engine
import logging


class Engine():
    """Common interface for all chess engines"""

    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        logging.error("Not implemented by this engine")

    def analyse(self, board: chess.Board) -> list[ScoredMove]:
        logging.error("Not implemented by this engine")

TODO: Document this

In [ ]:
from chess.engine import Score, Cp
from typing import Tuple, Optional
import math


class HighestScoreType(Score):

    def _score_tuple(self) -> Tuple[bool, bool, bool, int, Optional[int]]:
        return (True, True, True, math.inf, math.inf)

    def mate(self) -> None:
        return None

    def score(self, *, mate_score: Optional[int] = None) -> int:
        raise NotImplementedError

    def wdl(self, *, model="sf", ply: int = 30):
        raise NotImplementedError

    def __neg__(self) -> Cp:
        return LowestScoreType()

    def __pos__(self) -> Cp:
        raise NotImplementedError

    def __abs__(self) -> Cp:
        raise NotImplementedError


HighestScore = HighestScoreType()


class LowestScoreType(Score):

    def _score_tuple(self) -> Tuple[bool, bool, bool, int, Optional[int]]:
        return (False, False, False, -math.inf, math.inf)

    def mate(self) -> None:
        return None

    def score(self, *, mate_score: Optional[int] = None) -> int:
        raise NotImplementedError

    def wdl(self, *, model="sf", ply: int = 30):
        raise NotImplementedError

    def __neg__(self) -> Cp:
        return HighestScoreType()

    def __pos__(self) -> Cp:
        raise NotImplementedError

    def __abs__(self) -> Cp:
        raise NotImplementedError


LowestScore = LowestScoreType()

In [ ]:
from chess.engine import Mate, Cp, MateGiven

LowestScore < Mate(-0) < Mate(-1) < Cp(-50) < Cp(200) < Mate(4) < Mate(
    1
) < MateGiven < HighestScore